# 네이버 현재 상영작 데이터 수집
- settings.py : robots.txt 규칙을 따르지 않고 데이터 수집하도록 설정

In [6]:
## 1. 프로젝트 설정

In [12]:
!scrapy startproject naver_movie

New Scrapy project 'naver_movie', using template directory '/opt/anaconda3/lib/python3.8/site-packages/scrapy/templates/project', created in:
    /Users/dokyum/Workspace/DSS - /TIL-data-science/scrapy/naver_movie

You can start your first spider with:
    cd naver_movie
    scrapy genspider example example.com


In [61]:
!tree naver_movie/

naver_movie/
├── movie.csv
├── naver_movie
│   ├── __init__.py
│   ├── __pycache__
│   │   ├── __init__.cpython-38.pyc
│   │   ├── items.cpython-38.pyc
│   │   └── settings.cpython-38.pyc
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       ├── __pycache__
│       │   ├── __init__.cpython-38.pyc
│       │   └── spiders.cpython-38.pyc
│       └── spiders.py
└── scrapy.cfg

4 directories, 14 files


In [7]:
## 2. xpath 찾기

In [13]:
import scrapy
import requests
import re

from scrapy.http import TextResponse

In [16]:
headers= {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
req = requests.get("https://movie.naver.com/movie/running/current.nhn", headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://movie.naver.com/movie/running/current.nhn>

In [26]:
links = response.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href').extract()
links = [ response.urljoin(link) for link in links]
len(links), links[0]

(98, 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=163834')

In [27]:
### 상세 데이터 xpath : title, count, star
link = links[0]

In [28]:
req = requests.get(link, headers=headers)
response = TextResponse(req.url, body=req.text, encoding='utf-8')
response

<200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=163834>

In [47]:
title = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()').extract()[0]
count = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()').extract()[0].split('명')[0]
star1 = response.xpath('//*[@id="actualPointPersentBasic"]/div/em/text()').extract()
star1 = ''.join(star1)

star2 = response.xpath('//*[@id="actualPointPersentBasic"]/div/span/span/text()').extract()[0]
star2 = re.findall('[0-9]{1,2}\.[0-9]{2}', star2)[0]

title, count, star2

('원더 우먼 1984', '461,434', '7.61')

In [48]:
## 3. items. py

In [ ]:
# %load naver_movie/naver_movie/items.py

In [50]:
%%writefile naver_movie/naver_movie/items.py

import scrapy


class NaverMovieItem(scrapy.Item):
    title = scrapy.Field()
    count = scrapy.Field()
    star = scrapy.Field()


Overwriting naver_movie/naver_movie/items.py


In [9]:
## 4. spider.py

In [62]:
%%writefile naver_movie/naver_movie/spiders/spiders.py
import scrapy
from naver_movie.items import NaverMovieItem

class NaverSpider(scrapy.Spider):
    name = "NaverMovie"
    allow_domain = ['https://movie.naver.com/']
    start_urls = ['https://movie.naver.com/movie/running/current.nhn']
    
    def parse(self, response):
        links = response.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href').extract()
        links = [ response.urljoin(link) for link in links]
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):
        item = NaverMovieItem()
        item['title'] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()').extract()[0]
        try:
            item['count'] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()').extract()[0].split('명')[0]
        except:
            item['count'] = "0명"
        star1 = response.xpath('//*[@id="actualPointPersentBasic"]/div/em/text()').extract()
        item['star'] = ''.join(star1)
        yield item


Overwriting naver_movie/naver_movie/spiders/spiders.py


In [10]:
## 5. scrapy crawl  : 403 error 발생 : robots.txt

In [63]:
%%writefile run.sh
cd naver_movie
scrapy crawl NaverMovie -o movie.csv

Overwriting run.sh


In [54]:
!/bin/bash run.sh

2021-01-04 20:17:52 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: naver_movie)
2021-01-04 20:17:52 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.3 (default, Jul  2 2020, 11:26:31) - [Clang 10.0.0 ], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform macOS-10.15.7-x86_64-i386-64bit
2021-01-04 20:17:52 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-01-04 20:17:52 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'naver_movie',
 'NEWSPIDER_MODULE': 'naver_movie.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['naver_movie.spiders']}
2021-01-04 20:17:52 [scrapy.extensions.telnet] INFO: Telnet Password: ed77da298663f895
2021-01-04 20:17:52 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 's

In [11]:
## 6. settings.py 
# ROBOTSTXT_OBEY = False

In [55]:
## 7. scrapy crawl

In [64]:
!/bin/bash run.sh

2021-01-04 20:25:17 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: naver_movie)
2021-01-04 20:25:17 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.3 (default, Jul  2 2020, 11:26:31) - [Clang 10.0.0 ], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform macOS-10.15.7-x86_64-i386-64bit
2021-01-04 20:25:17 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-01-04 20:25:17 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'naver_movie',
 'NEWSPIDER_MODULE': 'naver_movie.spiders',
 'SPIDER_MODULES': ['naver_movie.spiders']}
2021-01-04 20:25:17 [scrapy.extensions.telnet] INFO: Telnet Password: 4c8f2ad396a5a992
2021-01-04 20:25:17 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexpo

2021-01-04 20:25:19 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=182525>
{'count': '0명', 'star': '9.57', 'title': '언플랜드'}
2021-01-04 20:25:19 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=189141>
{'count': '0명', 'star': '9.04', 'title': '삼진그룹 영어토익반'}
2021-01-04 20:25:19 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=192102>
{'count': '0명', 'star': '8.83', 'title': '운디네'}
2021-01-04 20:25:19 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=155123> (referer: https://movie.naver.com/movie/running/current.nhn)
2021-01-04 20:25:19 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=10286> (referer: https://movie.naver.com/movie/running/current.nhn)
2021-01-04 20:25:19 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/

2021-01-04 20:25:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=192191>
{'count': '0명', 'star': '8.08', 'title': '도망친 여자'}
2021-01-04 20:25:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=38723>
{'count': '0명', 'star': '9.20', 'title': '꽁치의 맛'}
2021-01-04 20:25:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=19222>
{'count': '0명', 'star': '', 'title': '공작 부인'}
2021-01-04 20:25:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=38720>
{'count': '0명', 'star': '8.00', 'title': '가을 햇살'}
2021-01-04 20:25:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=185282>
{'count': '0명', 'star': '9.15', 'title': '타오르는 여인의 초상'}
2021-01-04 20:25:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=11

2021-01-04 20:25:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=198430> (referer: https://movie.naver.com/movie/running/current.nhn)
2021-01-04 20:25:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=188431> (referer: https://movie.naver.com/movie/running/current.nhn)
2021-01-04 20:25:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=187848> (referer: https://movie.naver.com/movie/running/current.nhn)
2021-01-04 20:25:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=187845> (referer: https://movie.naver.com/movie/running/current.nhn)
2021-01-04 20:25:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=189623> (referer: https://movie.naver.com/movie/running/current.nhn)
2021-01-04 20:25:21 [scrapy.core.scraper] DEBUG: Scraped from <20

2021-01-04 20:25:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=195687>
{'count': '0명', 'star': '9.49', 'title': '페이트 스테이 나이트 헤븐즈필 III.스프링 송'}
2021-01-04 20:25:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=188447>
{'count': '0명', 'star': '8.67', 'title': '호프'}
2021-01-04 20:25:22 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=200214> (referer: https://movie.naver.com/movie/running/current.nhn)
2021-01-04 20:25:22 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=190722> (referer: https://movie.naver.com/movie/running/current.nhn)
2021-01-04 20:25:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=193331>
{'count': '0명', 'star': '9.39', 'title': '내언니전지현과 나'}
2021-01-04 20:25:22 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://m

In [65]:
!cd naver_movie/ && ls

movie.csv   naver_movie scrapy.cfg


In [67]:
df = pd.read_csv('./naver_movie/movie.csv')
df.tail()

,count,star,title
95,"427,241",8.34,이웃사촌
96,"59,799",9.13,화양연화
97,"1,530,885",8.50,도굴
98,"22,348",NaN,뱅가드
99,"195,434",8.23,조제


## awk
- 대용량의 문자열 데이터를 특정 패턴으로 찾거나 연상해서 결과를 출력해주는 문법(unix 계열)
- sed : 문자열을 찾아서 변환해주는 역할

In [83]:
!head -n 20 naver_movie/naver_movie/settings.py | tail -n 2

# Obey robots.txt rules
ROBOTSTXT_OBEY = True


In [82]:
!sed -i '' 's/ROBOTSTXT_OBEY = False/ROBOTSTXT_OBEY = True/' naver_movie/naver_movie/settings.py

In [81]:
!cd naver_movie/ && cd naver_movie/ && ls

__init__.py    items.py       pipelines.py   spiders
__pycache__    middlewares.py settings.py
